In [1]:
!python -m pip install pandas

In [2]:
!python -m pip install openpyxl


In [3]:
import os
import re

base_path = r"G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey"

# Month ordering
month_order = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# Patterns
month_pattern = re.compile(
    r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)",
    re.IGNORECASE
)
year_pattern = re.compile(r"(20\d{2})")

# Detect year folders from drive
year_folders = [
    f for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f)) and f.isdigit()
]

print("Detected year folders:", sorted(year_folders))

inventory = {}

for year in sorted(year_folders):
    year_path = os.path.join(base_path, year)

    # Accept both CSV and XLSX
    data_files = [
        f for f in os.listdir(year_path)
        if f.lower().endswith(".csv") or f.lower().endswith(".xlsx")
    ]

    inventory[year] = {}

    for file in data_files:
        upper = file.upper()

        # Detect type
        if upper.endswith(".XLSX"):
            filetype = "metadata"  # XLSX = metadata
        else:
            filetype = "survey"    # CSV = survey

        # Detect month
        month_match = month_pattern.search(upper)
        month = (
            month_match.group(1).capitalize()
            if month_match
            else "Unmatched"
        )

        # Detect year inside filename
        year_match = year_pattern.search(upper)
        file_year = year_match.group(1) if year_match else "UNKNOWN"

        # Store into inventory
        if month not in inventory[year]:
            inventory[year][month] = []

        inventory[year][month].append({
            "filename": file,
            "filetype": filetype,
            "file_year": file_year
        })

# Print clean summary
print("\n=== DATASET INVENTORY SUMMARY ===\n")

for yr in sorted(inventory.keys()):
    print(f"Year {yr}:")

    sorted_months = sorted(
        inventory[yr].keys(),
        key=lambda m: month_order.get(m, 99)
    )

    for month in sorted_months:
        print(f"  {month}:")
        for item in inventory[yr][month]:
            print(f"    {item['filename']} ({item['filetype']})")

    print()


Detected year folders: ['2018', '2019', '2022', '2023', '2024']

=== DATASET INVENTORY SUMMARY ===

Year 2018:
  January:
    JANUARY_2018.CSV (survey)
    JANUARY_2018_METADATA.xlsx (metadata)
  April:
    APRIL_2018.CSV (survey)
    APRIL_2018_METADATA.xlsx (metadata)
  July:
    JULY_2018.CSV (survey)
    JULY_2018_METADATA.xlsx (metadata)
  October:
    OCTOBER_2018.CSV (survey)
    OCTOBER_2018_METADATA.xlsx (metadata)

Year 2019:
  January:
    JANUARY_2019.CSV (survey)
    JANUARY_2019_METADATA.xlsx (metadata)
  April:
    APRIL_2019_METADATA.xlsx (metadata)
    APRIL_2019.CSV (survey)
  July:
    JULY_2019_METADATA.xlsx (metadata)
    JULY_2019.CSV (survey)
  October:
    OCTOBER_2019_METADATA.xlsx (metadata)
    OCTOBER_2019.CSV (survey)

Year 2022:
  January:
    JANUARY_2022.csv (survey)
    JANUARY_2022_METADATA.xlsx (metadata)
  February:
    FEBRUARY_2022.csv (survey)
    FEBRUARY_2022_METADATA.xlsx (metadata)
  March:
    MARCH_2022.csv (survey)
    MARCH_2022_METADATA.x

In [4]:
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load a dataset from the inventory.

    year: str, e.g., "2018"
    month: str, e.g., "January"
    filetype: "survey" or "metadata"
    sheet_number: 0(sheet 1) or 1(sheet 2)
    """
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])
    
    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)
    
    if sheet_number is not None:
        return pd.read_excel(file_path, sheet_name=sheet_number)
    
    return pd.read_excel(file_path)

In [5]:
import pandas as pd

# Load the second sheet of January 2018 metadata
second_sheet = load_dataset("2018", "January","metadata", 1)

# View the first few rows
second_sheet.head()


,PUFREG_VS1,Region,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,National Capital Region,13,NaN,NaN
1,NaN,NaN,Cordillera Administrative Region,14,NaN,NaN
2,NaN,NaN,Region I - Ilocos Region,1,NaN,NaN
3,NaN,NaN,Region II - Cagayan Valley,2,NaN,NaN
4,NaN,NaN,Region III - Central Luzon,3,NaN,NaN


In [6]:
# Load the second sheet of January 2018 metadata
second_sheet = load_dataset("2018", "January","metadata", 0)

# View the first few rows
second_sheet.head()


,QUEST,Questionnaire,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,_IDS0,(Id Items),NaN,NaN
1,NaN,NaN,NaN,NaN,PUFREG,Region
2,NaN,NaN,NaN,NaN,PUFPRV,Province
3,NaN,NaN,NaN,NaN,PUFPRRCD,Province Recode
4,NaN,NaN,NaN,NaN,PUFHHNUM,Household Unique Sequential Number


In [7]:
def extract_variables(df):
    """
    Extract variable names and descriptions from metadata DataFrame.
    Looks for values in columns 'Unnamed: 4' (variable) and 'Unnamed: 5' (description).
    Returns a clean DataFrame with ['Variable', 'Description'].
    """

    # Keep only rows where column 4 has a variable name
    filtered = df[df['Unnamed: 4'].notna()][['Unnamed: 4', 'Unnamed: 5']]

    # Rename columns
    filtered.columns = ['Variable', 'Description']

    return filtered


In [8]:
# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 0)

# Call your function
variables_df = extract_variables(January_metadata)

# View results
variables_df.head(50)


,Variable,Description
1,PUFREG,Region
2,PUFPRV,Province
3,PUFPRRCD,Province Recode
4,PUFHHNUM,Household Unique Sequential Number
6,PUFURB2K10,2010Urban-RuralFIES
7,PUFPWGTPRV,Final Weight Based on Projection (provincial p...
8,PUFSVYMO,Survey Month
9,PUFSVYYR,Survey Year
10,PUFPSU,Psu Number
11,PUFRPL,Replicate


In [9]:
import re
import pandas as pd

def process_metadata_sheet_binary1(df):
    """
    Strict horizontal reshape for metadata Sheet 2 (binary=1).

    Output columns:
      - Variable: header token like 'PUFREG_VS1'
      - Label: text immediately to the right of the variable in the header row
      - Category: category text (e.g., 'National Capital Region', 'Urban', '00 - 04')
      - Code: first numeric token after category (0 if none)
      - min_val: range minimum for range categories (0 if not a range)
      - max_val: range maximum for range categories (0 if not a range)
      - final_val: equals Code for category-code rows; 0 for ranges

    Notes:
      - No NaNs: text -> '', numbers -> 0
      - Handles merged cells, inconsistent positions, and header echo rows
      - Preserves strict horizontal alignment (no melting)
    """

    # Drop fully empty rows/columns and normalize index
    df = df.dropna(how="all").dropna(axis=1, how="all").reset_index(drop=True)

    # Helpers
    var_pat = re.compile(r"^[A-Z0-9_]+_VS\d+$")  # matches e.g., PUFREG_VS1
    def is_var(x):
        return isinstance(x, str) and bool(var_pat.match(x.strip()))

    def to_str(x):
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return ""
        return str(x).strip()

    def is_num_token(x):
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return False
        if isinstance(x, (int, float)) and not pd.isna(x):
            return True
        if isinstance(x, str):
            s = x.strip()
            return s.isdigit()
        return False

    def to_int(x, default=0):
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return default
        if isinstance(x, (int, float)) and not pd.isna(x):
            return int(x)
        if isinstance(x, str):
            s = x.strip()
            if s.isdigit():
                return int(s)
        return default

    def looks_like_range(text):
        if not isinstance(text, str):
            return False
        s = text.strip().lower()
        return ("-" in s) or ("and over" in s) or ("to" in s)

    def collect_nums_after(items, start_idx):
        nums = []
        for j in range(start_idx, len(items)):
            v = items[j]
            if is_num_token(v):
                nums.append(to_int(v))
        return nums

    records = []
    current_variable = ""
    current_label = ""

    # Iterate rows and maintain current variable/label context
    for _, row in df.iterrows():
        cells = [None if pd.isna(v) else v for v in row.tolist()]

        # Find a variable token in this row
        var_idx = next((i for i, v in enumerate(cells) if is_var(v)), None)
        if var_idx is not None:
            # Set variable
            current_variable = to_str(cells[var_idx])

            # Determine label: next non-empty, non-variable text to the right
            current_label = ""
            for j in range(var_idx + 1, len(cells)):
                val = cells[j]
                if isinstance(val, str) and val.strip() and not is_var(val):
                    current_label = val.strip()
                    break

            # Start scanning payload to the right of variable
            start_idx = var_idx + 1
        else:
            # Continue with existing context
            start_idx = 0

        # Build payload of non-empty cells from start_idx
        payload = []
        for j in range(start_idx, len(cells)):
            v = cells[j]
            if v is None:
                continue
            payload.append(v)

        if not payload:
            continue

        # Skip header echo rows: if first payload equals label, ignore row
        if current_label and to_str(payload[0]) == current_label:
            continue

        # Identify category: first non-variable text; fallback to first numeric if necessary
        cat_idx = None
        for j, v in enumerate(payload):
            if isinstance(v, str) and v.strip() and not is_var(v):
                cat_idx = j
                break
        if cat_idx is None:
            for j, v in enumerate(payload):
                if is_num_token(v):
                    cat_idx = j
                    break
        if cat_idx is None:
            continue

        category = to_str(payload[cat_idx])
        nums = collect_nums_after(payload, cat_idx + 1)

        # Decide mapping and fill fields
        min_val = max_val = code = final_val = 0

        if looks_like_range(category) and len(nums) >= 2:
            # Range case: take first two as min/max, code/final_val remain 0
            min_val, max_val = nums[0], nums[1]
            code = 0
            final_val = 0
        else:
            # Category-code case: first numeric after category is the code
            if len(nums) >= 1:
                code = nums[0]
                final_val = code
            # If category itself is numeric and no numeric after, treat category as label with code equal to itself
            elif category.isdigit():
                code = int(category)
                final_val = code

        # Record only if we have a variable and a category
        if current_variable and category:
            records.append({
                "Variable": current_variable,
                "Label": current_label,
                "Category": category,
                "Code": int(code),
                "min_val": int(min_val),
                "max_val": int(max_val),
                "final_val": int(final_val)
            })

    # Build output dataframe with strict schema; ensure no NaNs
    if not records:
        return pd.DataFrame(columns=["Variable", "Label", "Category", "Code", "min_val", "max_val", "final_val"])

    out = pd.DataFrame.from_records(records)
    for col in ["Variable", "Label", "Category"]:
        out[col] = out[col].fillna("")
    for col in ["Code", "min_val", "max_val", "final_val"]:
        out[col] = out[col].fillna(0).astype(int)
    return out


In [10]:
# Sheet 2 = binary 1 (category/value mappings)
sheet2_jan2018 = load_dataset(year="2018", month="January", filetype="metadata", sheet_number=1)
sheet2_jan2018_reshaped = process_metadata_sheet_binary1(sheet2_jan2018)
print("January 2018 (Sheet 2) preview:")
print(sheet2_jan2018_reshaped.head(30))

January 2018 (Sheet 2) preview:
          Variable                Label Category  Code  min_val  max_val  \
0   PUFURB2K10_VS1  2010Urban-RuralFIES    Urban     1        0        0   
1   PUFURB2K10_VS1  2010Urban-RuralFIES    Rural     2        0        0   
2     PUFSVYMO_VS1         Survey Month  January     1        0        0   
3     PUFSVYMO_VS1         Survey Month    April     4        0        0   
4     PUFSVYMO_VS1         Survey Month     July     7        0        0   
5     PUFSVYMO_VS1         Survey Month  October    10        0        0   
6    PUFHHSIZE_VS1       Household Size        1     1        0        0   
7    PUFHHSIZE_VS1       Household Size        2     2        0        0   
8    PUFHHSIZE_VS1       Household Size        3     3        0        0   
9    PUFHHSIZE_VS1       Household Size        4     4        0        0   
10   PUFHHSIZE_VS1       Household Size        5     5        0        0   
11   PUFHHSIZE_VS1       Household Size        6     6  

In [11]:
# Another month to prove that the code can scale/generalize
sheet2_apr2018 = load_dataset(year="2018", month="April", filetype="metadata", sheet_number=1)
sheet2_apr2018_reshaped = process_metadata_sheet_binary1(sheet2_apr2018)
print("\nApril 2018 (Sheet 2) preview:")
print(sheet2_apr2018_reshaped.head(30))


April 2018 (Sheet 2) preview:
          Variable                Label Category  Code  min_val  max_val  \
0   PUFURB2K10_VS1  2010Urban-RuralFIES    Urban     1        0        0   
1   PUFURB2K10_VS1  2010Urban-RuralFIES    Rural     2        0        0   
2     PUFSVYMO_VS1         Survey Month  January     1        0        0   
3     PUFSVYMO_VS1         Survey Month    April     4        0        0   
4     PUFSVYMO_VS1         Survey Month     July     7        0        0   
5     PUFSVYMO_VS1         Survey Month  October    10        0        0   
6    PUFHHSIZE_VS1       Household Size        1     1        0        0   
7    PUFHHSIZE_VS1       Household Size        2     2        0        0   
8    PUFHHSIZE_VS1       Household Size        3     3        0        0   
9    PUFHHSIZE_VS1       Household Size        4     4        0        0   
10   PUFHHSIZE_VS1       Household Size        5     5        0        0   
11   PUFHHSIZE_VS1       Household Size        6     6   